# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will past the accuracy of your model to the Coursera grader.

This notebook is tested in IBM Watson Studio under python 3.6

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by installing and importing everything we need for this exercise:

In [ ]:
!pip install --upgrade keras-applications keras-preprocessing setuptools tensorflow==1.14.0 keras==2.2.5

In [ ]:
import tensorflow
if not tensorflow.__version__ == '1.14.0':
    print(tensorflow.__version__)
    raise ValueError('please upgrade to TensorFlow 1.14.0, or restart your Kernel (Kernel->Restart & Clear Output)')

import keras
if not keras.__version__ == '2.2.5':
    print(keras.__version__)
    raise ValueError('please upgrade to Keras 2.2.5, or restart your Kernel (Kernel->Restart & Clear Output)')

IMPORTANT! => Please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from keras.datasets import reuters

max_words = 1000
#num_words: the maximum number of words to keep, based on word frequency. 
#Only the most common num_words-1 words will be kept.
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [4]:
#Understanding the dataset
# size = 0
# for p in x_train:
#     print(len(p))
#     size = size + len(p)
# print(size)
# print(len(x_train))
print(x_train)

[list([1, 56, 2, 141, 2, 71, 8, 16, 40, 200, 6, 438, 2, 806, 2, 81, 5, 2, 2, 2, 7, 10, 587, 7, 50, 261, 5, 2, 806, 33, 839, 79, 2, 69, 10, 147, 20, 128, 7, 4, 2, 49, 4, 49, 8, 16, 33, 57, 69, 78, 11, 79, 335, 21, 10, 2, 959, 503, 92, 4, 587, 16, 8, 92, 4, 270, 16, 33, 2, 2, 806, 31, 197, 13, 2, 16, 8, 2, 806, 189, 40, 365, 2, 2, 9, 363, 6, 2, 117, 124, 7, 89, 900, 2, 6, 2, 172, 2, 236, 7, 4, 37, 38, 9, 2, 17, 12])
 list([1, 99, 234, 60, 9, 752, 111, 8, 25, 544, 20, 324, 2, 2, 640, 56, 2, 323, 40, 385, 25, 73, 794, 220, 13, 69, 32, 251, 18, 15, 7, 197, 9, 19, 445, 18, 15, 7, 80, 2, 7, 10, 99, 98, 276, 13, 99, 234, 5, 69, 19, 451, 18, 15, 92, 131, 4, 49, 8, 4, 211, 33, 2, 2, 2, 22, 4, 293, 2, 218, 17, 12])
 list([1, 103, 74, 92, 39, 128, 2, 2, 11, 2, 14, 2, 11, 88, 160, 147, 35, 2, 14, 83, 272, 35, 349, 74, 90, 67, 83, 2, 14, 72, 119, 344, 275, 188, 2, 14, 12, 546, 17, 12])
 ...
 list([1, 486, 341, 147, 26, 14, 147, 26, 255, 219, 252, 68, 146, 91, 102, 17, 12])
 list([1, 56, 2, 697, 149,

In [5]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [6]:
print(x_train)
len(x_train)

[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]


8982

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [7]:
print(y_train,y_test)

[ 4  3  3 ...  3 25 11] [ 4 19  4 ...  3  1 43]


In [8]:
num_classes = 46
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print(y_train,y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [9]:
model = Sequential()   # Instantiate sequential model
model.add(Dense(512, activation='relu', input_shape=(max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(.5)) # Add second layer
model.add((Dense(num_classes, activation='softmax'))) # Add third layer

W0118 22:52:57.137340 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0118 22:52:57.163443 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0118 22:52:57.166302 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0118 22:52:57.191382 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.p

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [10]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

W0118 22:52:58.089131 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0118 22:52:58.129274 139945846683456 deprecation_wrapper.py:119] From /opt/conda/envs/Python36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [11]:
batch_size = 32
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

W0118 22:52:59.131344 139945846683456 deprecation.py:323] From /opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
8982/8982 [==============================] - 9s 1ms/step - loss: 1.3882 - acc: 0.6915
Epoch 2/5
8982/8982 [==============================] - 9s 1ms/step - loss: 0.7704 - acc: 0.8201
Epoch 3/5
8982/8982 [==============================] - 10s 1ms/step - loss: 0.5463 - acc: 0.8646
Epoch 4/5
8982/8982 [==============================] - 10s 1ms/step - loss: 0.4178 - acc: 0.8979
Epoch 5/5
8982/8982 [==============================] - 10s 1ms/step - loss: 0.3502 - acc: 0.9080


In [12]:
score = model.evaluate(x_test, y_test)

2246/2246 [==============================] - 0s 92us/step


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [13]:
score[1]

0.7943009795191451

In [14]:
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  0.8607072335529412
Test accuracy:  0.7943009795191451


Congratulations, now it's time to submit your result to the Coursera grader by executing the following cells (Programming Assingment, Week2). 

We have to install a little library in order to submit to coursera


In [15]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-01-18 22:55:07--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,540       --.-K/s   in 0s      

2020-01-18 22:55:07 (25.7 MB/s) - ‘rklib.py’ saved [2540/2540]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [16]:
from rklib import submit
import json

key = "XbAMqtjdEeepUgo7OOVwng"
part = "HCvcp"
email = 'giovani.machado@gmail.com'
token = "I18hxhDSlOmylaOr" #you can obtain it from the grader page on Coursera (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)


submit(email, token, 'XbAMqtjdEeepUgo7OOVwng', part, [part], json.dumps(score[1]*100))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"ozVf2","id":"tE4j0qhMEeecqgpT6QjMdA~ozVf2~plhLmTpFEeqNYwpBQTsAuw","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
